In [ ]:
import plotly.graph_objects as go
import numpy as np

In [ ]:
def _plotlyBox(mag=(0,0,1), pos = (0,0,0), dim = (10,10,10), angle=0, axis=(0,0,1), cst=0.1, **kwargs):
    box = go.Mesh3d(
        i = np.array([7, 0, 0, 0, 4, 4, 2, 6, 4, 0, 3, 7]),
        j = np.array([3, 4, 1, 2, 5, 6, 5, 5, 0, 1, 2, 2]),
        k = np.array([0, 7, 2, 3, 6, 7, 1, 2, 5, 5, 7, 6]),
    )
    x = np.array([-1, -1, 1, 1, -1, -1, 1, 1])*0.5*dim[0]+pos[0]
    y = np.array([-1, 1, 1, -1, -1, 1, 1, -1])*0.5*dim[1]+pos[1]
    z = np.array([-1, -1, -1, -1, 1, 1, 1, 1])*0.5*dim[2]+pos[2]
    
    if cst is not False:
        box.colorscale = _getColorscale(cst)
        box.intensity = _getIntensity(points=(x,y,z), mag=mag, pos=pos)
        
    if angle!=0:
        x,y,z = AxisRotation(points=[x,y,z], angle=angle, axis=axis, anchor=pos)
    
    box.x , box.y, box.z = x,y,z
    box.update(**kwargs)
    return box

def _plotlyCylinder(mag=(0,0,1), pos = (0,0,0), dim = (5,10,0), angle=0, axis=(0,0,1), cst=False, **kwargs):
    dim=np.array(dim)
    if len(dim)==2:
        dim = np.array(list(dim[0:2]) + [0])
    elif len(dim) == 3 and dim[2]==0:
        dim[2] = 1e-5
    ri = min(dim[0]/2,dim[2]/2)
    ro = max(dim[0]/2,dim[2]/2)
    hmin, hmax = -dim[1]/2, dim[1]/2
     
    h = [hmin,hmin,hmax,hmax,hmin]
    s = np.linspace(0, 2 * np.pi, 40)
    sa, ha = np.meshgrid(s, h)

    ro = dim[0]/2  ; ri = dim[2]/2
    x = ro * np.cos(sa)
    y = ro * np.sin(sa)
    z = ha

    x[0] = x[-2] = x[-1] = ri*np.cos(s)
    y[0] = y[-2] = y[-1] = ri*np.sin(s)
    
    cylinder=go.Surface(x=x+pos[0], y=y+pos[1], z=z+pos[2])
    if cst is not False:
        cylinder.colorscale = _getColorscale(cst)
        cylinder.surfacecolor = _getIntensity(points=(x,y,z), mag=mag, pos=(0,0,0))
    if angle!=0:
        xr,yr,zr = AxisRotation(points=np.array([x.flatten(),y.flatten(),z.flatten()]), angle=angle, axis=axis, anchor=pos)
        cylinder.update(x=xr.reshape(x.shape), y=yr.reshape(y.shape), z=zr.reshape(z.shape))
    
    cylinder.update(**kwargs)
    return cylinder

def AxisRotation(points,angle,axis,anchor):
    from scipy.spatial.transform import Rotation
    points= np.array(points)
    rotation = Rotation.from_rotvec(np.deg2rad(angle)*np.array(axis))  
    box_rotated = rotation.apply(points.T-anchor) + anchor
    return box_rotated.T

def _getIntensity(points, mag, pos):
    '''points: [x,y,z] array'''
    p = np.array(points)
    pos = np.array(pos)
    m = np.array(mag) / np.linalg.norm(mag)
    a = ((p[0]-pos[0])*m[0] + (p[1]-pos[1])*m[1] + (p[2]-pos[2])*m[2])
    b = (p[0]-pos[0])**2 + (p[1]-pos[1])**2 + (p[2]-pos[2])**2
    return a / np.sqrt(b)

def _getColorscale(cst=0.1):
    return [[0, 'turquoise'], [0.5*(1-cst), 'turquoise'],[0.5*(1+cst), 'magenta'], [1, 'magenta']]

In [ ]:
fig = go.FigureWidget()
fig.add_trace(_plotlyCylinder(mag=(1,0,0),dim=(40,30,20), pos=(20,70,40), cst=0.1))
#fig.add_trace(_plotlyBox(mag=(0,0,1), dim=(10,5,5), pos=(10,0,0), angle=50, axis=(0,1,1), cst=0.1))
fig